In [6]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain_groq import ChatGroq

In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
KEY = os.getenv("GROQ_API_KEY")

In [10]:
llm = ChatGroq(temperature=0.5,groq_api_key=KEY,model='mixtral-8x7b-32768')

In [11]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000170D187AE20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000170D188A9A0>, temperature=0.5)

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_community.callbacks import get_openai_callback
import PyPDF2
import groq


In [13]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "answer",
    },
}

In [59]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quize of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
{response_json}

"""

In [60]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [61]:
quize_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [62]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity 
if the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [63]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [65]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [66]:
generate_evaluate_chain = SequentialChain(chains=[quize_chain,review_chain], input_variables=["text","number","subject","tone","response_json"],output_variables=['quiz','review'],verbose=True)

In [67]:
file_path = r"E:\MCQgen\data.txt"

In [68]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [69]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of at least one cell that processes hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other s

In [70]:
# serializing the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "answer"}}'

In [71]:
NUMBER=3
SUBJECT="biology"
TONE="simple"

In [81]:
# how to setup token usage tracking in LangChain
from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of at least one cell that processes hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that 

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity 
if the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{
"1": {
"mcq": "What is the cell theory's third tenet that was widely accepted by the 1860s?",
"options": {
"a": "The basic unit of organisms is the tissue.",
"b": "Individual cells have some characteristics of life.",
"c": "All cells come from the division of other cells.",
"d": "Cells are important for the function of organs."
},
"correct": "c"
},
"2": {
"mcq": "Who is considered the father of taxonomy for his work in the 1735 and 1750s?",
"options": {
"a": "Carl Linnaeus",
"b": "Georges-Louis Leclerc, Comte de Buffon",
"c": "Jean-Baptiste Lamarck",
"d": "Charles Darwin"
},
"correct

In [82]:
print(f"Total Token : {cb.total_tokens}")
print(f"Prompt Token : {cb.prompt_tokens}")
print(f"Completion Token : {cb.completion_tokens}")
print(f"Total Cost : {cb.total_cost}")

Total Token : 2496
Prompt Token : 2103
Completion Token : 393
Total Cost : 0.0


In [103]:
quiz = response.get('quiz')

In [104]:
quiz = json.loads(quiz)

In [106]:
quiz

{'1': {'mcq': "What is the cell theory's third tenet that was widely accepted by the 1860s?",
  'options': {'a': 'The basic unit of organisms is the tissue.',
   'b': 'Individual cells have some characteristics of life.',
   'c': 'All cells come from the division of other cells.',
   'd': 'Cells are important for the function of organs.'},
  'correct': 'c'},
 '2': {'mcq': 'Who is considered the father of taxonomy for his work in the 1735 and 1750s?',
  'options': {'a': 'Carl Linnaeus',
   'b': 'Georges-Louis Leclerc, Comte de Buffon',
   'c': 'Jean-Baptiste Lamarck',
   'd': 'Charles Darwin'},
  'correct': 'a'},
 '3': {'mcq': "Which scientist's work laid the foundation for modern genetics in 1865?",
  'options': {'a': 'Gregor Mendel',
   'b': 'Alfred Hershey',
   'c': 'Martha Chase',
   'd': 'James Watson'},
  'correct': 'a'}}

In [105]:
quiz_table_data=[]

for key, value in quiz.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [107]:
quiz = pd.DataFrame(quiz_table_data)

In [108]:
quiz.to_csv('Biology_quiz.csv',index=False)